<a href="https://colab.research.google.com/github/andromeda123248/Dip/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
! pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0829329772ccf06a4db7a21c0768774de95a468c56ad07a8e6e17bae834b2824
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import re

In [4]:
import nltk
nltk.download('punkt')

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
! pip install faiss-gpu ##Installing GPU version of faiss



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.3 MB/s eta 0:00:00


In [6]:
! pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [7]:
! pip install sentence_transformers ## For textual similarity, using pretrained models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [8]:
import faiss
from sentence_transformers import SentenceTransformer, util

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Load the movies data

In [10]:
json_file_path = 'ex_tr'

df = pd.read_json(json_file_path)

df[['name', 'description', 'genres']] = df['docs'].apply(
        lambda x: pd.Series([x['name'], x['description'], ', '.join([genre['name'] for genre in x['genres']])]))

df = df.drop(columns=['docs'])

print(df.head())

                      name                                        description  \
0  Там, где танцуют стерхи  Восьмилетний Ваня переживает смерть матери. Он...   
1                     Паук  Старшеклассника Паркера кусает радиоактивный п...   
2                   Карина  В конце лета 2014 года, перед наступлением хол...   
3                 Приговор  2009 год. Небольшой городок на берегу Иссык-Ку...   
4                   Джойка  Джой Уомак — одна из немногих американок, кото...   

                       genres  
0                       драма  
1      короткометражка, ужасы  
2  биография, детектив, драма  
3              драма, триллер  
4            драма, биография  


In [11]:
df['id']=df.index

In [12]:
sentences = df['description'].tolist()
print("Number of Sentences in Movie Description ",len(sentences))

Number of Sentences in Movie Description  1500


In [13]:
embeddings = model.encode(sentences)
faiss.normalize_L2(embeddings)

In [14]:
print("Shape of the EMbeddings is ",embeddings.shape)

Shape of the EMbeddings is  (1500, 768)


In [15]:
dim=768
ncentroids=50
m=16
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, m , 8)
index.train(embeddings)
print(index.is_trained)
faiss.write_index(index, "trained.index")

True


In [16]:
ids = df['id'].tolist()
ids = np.array(ids)
index.add_with_ids(embeddings,ids)
print(index.ntotal)

1500


In [17]:
faiss.write_index(index,"block.index")

In [18]:
def searchFAISSIndex(data,id_col_name,query,index,nprobe,model,topk=20):
    query_embedding=model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)


    index.nprobe=nprobe

    D,I=index.search(query_embedding,topk)
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [19]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

In [26]:
query="история про успешного певца"
search_result=searchFAISSIndex(df,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','description','name','cosine_sim','L2_score']]

In [27]:
search_result

,id,description,name,cosine_sim,L2_score
5,366,"История гитариста Эммета Рэя, о его любовных п...",Сладкий и гадкий,0.867984,0.513841
16,1145,Флоренс Фостер Дженкинс всегда мечтала петь на...,Примадонна,0.840066,0.565570
14,936,"История Иэна Кертиса, фронтмена британской пос...",Контроль,0.828053,0.586424
6,397,История жизни пианиста-виртуоза Дэвид Хельфгот...,Блеск,0.791729,0.645400
13,884,Журналист Стив Лопес встречает Натаниеля Айерс...,Солист,0.761346,0.690874
15,978,У каждого в жизни была минута славы. Или обяза...,Мечты сбываются!,0.758530,0.694939
0,10,Эта музыкальная феерия состоит из самых ярких ...,Бетт Мидлер: Шоу должно продолжаться,0.740203,0.720829
4,277,Замкнутый и одинокий Трэвис берется помогать т...,Стендап по дружбе,0.739379,0.721970
2,81,История жизни известной певицы и активистки фе...,Я — женщина,0.732211,0.731831
11,794,"Стремительный ритм, виртуозная и озорная импро...",Зимний вечер в Гаграх,0.725623,0.740779
